## NYC Taxi Data Experimnt Tracking

In [2]:
!python -V

Python 3.11.5


In [1]:
import subprocess
import time
from IPython.display import IFrame
from sklearn.model_selection import train_test_split, GridSearchCV
from tqdm import tqdm



In [2]:
import requests
import pickle
import numpy as np

import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVR



In [5]:
train_raw_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')
val_raw_data = pd.read_parquet('data/green_tripdata_2024-04.parquet')

In [6]:
train_raw_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-03-01 00:10:52,2024-03-01 00:26:12,N,1.0,129,226,1.0,1.72,12.8,1.0,0.5,3.06,0.0,NaN,1.0,18.36,1.0,1.0,0.00
1,2,2024-03-01 00:22:21,2024-03-01 00:35:15,N,1.0,130,218,1.0,3.25,17.7,1.0,0.5,0.00,0.0,NaN,1.0,20.20,2.0,1.0,0.00
2,2,2024-03-01 00:45:27,2024-03-01 01:04:32,N,1.0,255,107,2.0,4.58,23.3,1.0,0.5,3.50,0.0,NaN,1.0,32.05,1.0,1.0,2.75
3,1,2024-03-01 00:02:00,2024-03-01 00:23:45,N,1.0,181,71,1.0,0.00,22.5,0.0,1.5,0.00,0.0,NaN,1.0,24.00,1.0,1.0,0.00
4,2,2024-03-01 00:16:45,2024-03-01 00:23:25,N,1.0,95,135,1.0,1.15,8.6,1.0,0.5,1.00,0.0,NaN,1.0,12.10,1.0,1.0,0.00


In [7]:
def process_dataframe(data):
    data.lpep_dropoff_datetime = pd.to_datetime(data.lpep_dropoff_datetime)
    data.lpep_pickup_datetime = pd.to_datetime(data.lpep_pickup_datetime)

    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data.duration = data.duration.apply(lambda td: td.total_seconds() / 60)
    data = data[(data.duration >= 1) & (data.duration <= 60)] # 
    
    data['PULocationID'].astype(str, copy=False)
    data['DOLocationID'].astype(str, copy=False)
    
    return data

In [8]:
num_features = ['trip_distance', 'extra', 'fare_amount']
cat_features = ['PULocationID', 'DOLocationID']

In [9]:
X_train = process_dataframe(train_raw_data)[num_features + cat_features]
X_val = process_dataframe(val_raw_data)[num_features + cat_features] 

y_train = process_dataframe(train_raw_data)['duration']
y_val = process_dataframe(val_raw_data)['duration'] 

In [10]:
X_val.isnull().sum()

trip_distance    0
extra            0
fare_amount      0
PULocationID     0
DOLocationID     0
dtype: int64

## Simple Experiment

In [11]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.398267721662138

In [12]:
np.mean(y_pred)

14.131639061800543

## MLflow tracking

In [13]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [14]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("green_taxi")

<Experiment: artifact_location='file:///C:/Users/user/PycharmProjects/qwerty/mlruns/1', creation_time=1721082632454, experiment_id='1', last_update_time=1721082632454, lifecycle_stage='active', name='green_taxi', tags={}>

In [15]:
sqlite_db_path = "sqlite:///mlflow.db"


In [16]:
pwd

'C:\\Users\\user\\PycharmProjects\\qwerty'

In [19]:
with mlflow.start_run() as run:
    mlflow.set_tag("workspace", "inclass")
    mlflow.log_param("model_name", "Lasso")
    mlflow.log_param("train_data", "green_tripdata_2024-03.parquet")
    
    alpha = 0.1
    lr = Lasso(alpha=alpha)
    mlflow.log_param("alpha", alpha)
    
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred)
    print('rmse:', rmse)
    mlflow.log_metric("rmse", rmse)
    print(f"Run ID: {run.info.run_id}")

    mlflow.end_run()
print('Experiment recorded (chunk executed)')

rmse: 29.148545881396135
Run ID: 1573793f0aab498dbf4127637a5d2554
Experiment recorded (chunk executed)


In [20]:
import logging
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("green_taxi")
logging.basicConfig(level=logging.DEBUG)


In [17]:
import os

# List the contents of the 'mlruns' directory
mlruns_path = 'mlruns'
if os.path.exists(mlruns_path):
    contents = os.listdir(mlruns_path)
    print("Contents of 'mlruns':")
    for item in contents:
        print(item)
else:
    print(f"The directory '{mlruns_path}' does not exist.")


Contents of 'mlruns':
.trash
0
461779665582921331
models


In [18]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('mlflow.db')

# Create a cursor object
cur = conn.cursor()

# Execute a query to check the tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch and print the results
tables = cur.fetchall()
print("Tables in the database:", tables)
query = "SELECT * FROM experiments"
experiments_df = pd.read_sql_query(query, conn)

print('query res:', experiments_df)
# Close the connection
conn.close()


Tables in the database: [('experiments',), ('alembic_version',), ('experiment_tags',), ('tags',), ('registered_models',), ('runs',), ('registered_model_tags',), ('model_version_tags',), ('model_versions',), ('latest_metrics',), ('metrics',), ('registered_model_aliases',), ('datasets',), ('inputs',), ('input_tags',), ('params',), ('trace_info',), ('trace_tags',), ('trace_request_metadata',)]
query res:    experiment_id        name  \
0              0     Default   
1              1  green_taxi   

                                   artifact_location lifecycle_stage  \
0                                mlflow-artifacts:/0          active   
1  file:///C:/Users/user/PycharmProjects/qwerty/m...          active   

   creation_time  last_update_time  
0  1721069481320     1721069481320  
1  1721082632454     1721082632454  


In [ ]:
### ------------- XGBOOST -----------


In [19]:
from xgboost import XGBRegressor


In [20]:
with mlflow.start_run() as run:
    mlflow.set_tag("workspace", "inclass")
    mlflow.log_param("model_name", "XGBoost")
    mlflow.log_param("train_data", "green_tripdata_2024-03.parquet")
    
    # Set parameters for XGBoost
    params = {
        "objective": "reg:squarederror",
        "learning_rate": 0.1,
        "max_depth": 6,
        "n_estimators": 100
    }
    
    # Log the parameters
    for param, value in params.items():
        mlflow.log_param(param, value)
    
    # Train the model
    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    # Calculate RMSE
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print('rmse xgboost-1', rmse)
    mlflow.log_metric("rmse", rmse)
    
    # Log the model
    mlflow.xgboost.log_model(model, "model")
    
    # Print run_id for verification
    print(f"Run ID: {run.info.run_id}")

print('Experiment-2 recorded (chunk executed)')


C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:15:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


rmse xgboost-1 3.0408032407468317
Run ID: d9bf5d20894141b2b786d45498825a54
Experiment-2 recorded (chunk executed)


In [21]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBRegressor, DMatrix, train as xgb_train



In [22]:
# Define the parameter grid for GridSearchCV
# Define the hyperparameter space
space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'max_depth': hp.choice('max_depth', [4, 6, 8]),
    'gamma': hp.loguniform('gamma', -3, 0),  # loguniform parameter
    'num_boost_round': hp.choice('num_boost_round', [100, 200, 300]),
    'early_stopping_rounds': hp.choice('early_stopping_rounds', [10, 20, 30])
}

In [23]:
# Define the objective function
def objective(params):
    with mlflow.start_run() as run:
        mlflow.set_tag("workspace", "inclass")
        mlflow.log_param("model_name", "XGBoost")
        mlflow.log_param("train_data", "green_tripdata_2024-03.parquet")
        
        # Split the data into DMatrix format for XGBoost
        dtrain = DMatrix(X_train, label=y_train)
        dval = DMatrix(X_val, label=y_val)
        
        # Extract the parameters for XGBoost
        xgb_params = {key: params[key] for key in params if key not in ['num_boost_round', 'early_stopping_rounds']}
        
        # Train the model with early stopping
        evals = [(dval, 'eval'), (dtrain, 'train')]
        model = xgb_train(
            xgb_params, 
            dtrain, 
            num_boost_round=params['num_boost_round'], 
            evals=evals, 
            early_stopping_rounds=params['early_stopping_rounds'],
            verbose_eval=False
        )
        
        # Predict on the validation set using the best iteration
        best_iteration = model.best_iteration if model.best_iteration is not None else params['num_boost_round']
        y_pred = model.predict(dval, iteration_range=(0, best_iteration))
        
        # Calculate RMSE
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
        # Log the model
        mlflow.xgboost.log_model(model, "model")
        
        # Log the parameters
        for param, value in params.items():
            mlflow.log_param(param, value)
        
        # Print run_id for verification
        print(f"Run ID: {run.info.run_id}, RMSE: {rmse}")

        return {'loss': rmse, 'status': STATUS_OK}

# Perform hyperparameter optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

print("Best parameters:", best)

print('Hyperparameter optimization and experiment logging completed')

  0%|                                                                                                                   | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:15:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)



Run ID: bce7ec135342427786be774b6357f0b3, RMSE: 3.01484313959433                                                                                               
  2%|█▊                                                                                         | 1/50 [00:02<02:12,  2.70s/trial, best loss: 3.01484313959433]

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\user

Run ID: d492a82470a54215a57f0a70e7add3a0, RMSE: 3.232038294231299                                                                                              
  4%|███▋                                                                                       | 2/50 [00:05<02:06,  2.63s/trial, best loss: 3.01484313959433]

...

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\user

Run ID: 98f8bc29d0a240e280d0f6c79ea8ba83, RMSE: 2.991871461768265                                                                                              
 98%|██████████████████████████████████████████████████████████████████████████████████████▏ | 49/50 [02:31<00:03,  3.23s/trial, best loss: 2.9754679365078545]

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

C:\Users\user

Run ID: d702000c1a994b9e8ea32fb2bc4c1320, RMSE: 3.2453762065109095                                                                                             
100%|████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:34<00:00,  3.10s/trial, best loss: 2.9754679365078545]
Best parameters: {'early_stopping_rounds': 1, 'gamma': 0.09075014030075113, 'learning_rate': 0.04447507493826804, 'max_depth': 2, 'num_boost_round': 2}
Hyperparameter optimization and experiment logging completed


C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

C:\Users\user\PycharmProjects\qwerty\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



### Load model

In [18]:
logged_model_id = 'runs:/d2c5d41c65524388bdd18e8ad385317d/model'
loaded_model = mlflow.pyfunc.load_model(logged_model_id)

In [19]:
type(loaded_model)

mlflow.pyfunc.PyFuncModel

In [21]:
from mlflow import MlflowClient


In [23]:
client = MlflowClient()
client.get_experiment_by_name('green_taxi')


<Experiment: artifact_location='file:///C:/Users/user/PycharmProjects/qwerty/mlruns/1', creation_time=1721082632454, experiment_id='1', last_update_time=1721082632454, lifecycle_stage='active', name='green_taxi', tags={}>

In [24]:
runs = client.search_runs(experiment_ids='1')

In [25]:
len(runs)

54

In [27]:
for run in runs:
    print(f"run_id = {run.info.run_id}, rmse = {run.data.metrics}")

run_id = d702000c1a994b9e8ea32fb2bc4c1320, rmse = {'rmse': 3.2453762065109095}
run_id = 98f8bc29d0a240e280d0f6c79ea8ba83, rmse = {'rmse': 2.991871461768265}
run_id = 376b588b186e489095698d5c6d2dfd2a, rmse = {'rmse': 3.003389292754948}
run_id = 101690934d1f4d17ab979cf9137ed7e9, rmse = {'rmse': 3.0130361865028696}
run_id = f5c5c5cc3fe64c00a3406b47423c1a1f, rmse = {'rmse': 3.0167878076877908}
run_id = 6e8ae133769e4d11ae8c9e582fd80ab5, rmse = {'rmse': 3.0575644569719236}
run_id = 2379f7698711481ab8c3fb4955394708, rmse = {'rmse': 3.3811975978429474}
run_id = c1674f4dc9f44ed58e58255f1bf2b49f, rmse = {'rmse': 3.032297771060551}
run_id = 8e0cfe8db0bb44bbb52b410a5d3821ff, rmse = {'rmse': 3.0065371971354335}
run_id = 7697b74d3fe84847b5823895d57e7c5f, rmse = {'rmse': 3.005297152585824}
run_id = 39371c97d2f844fb8334040fe1d4ccb9, rmse = {'rmse': 3.277196016766832}
run_id = df5325ca753b468d99ec4443def49044, rmse = {'rmse': 2.9992501443896824}
run_id = 0442684afe664b3c9e2f25a43f74ebab, rmse = {'rmse'

In [39]:
# Filter runs with rmse < thr (threshold)
thr = 2.99
filtered_runs = []
for run in runs:
    metrics = run.data.metrics
    if 'rmse' in metrics and metrics['rmse'] < thr:
        filtered_runs.append({
            'run_id': run.info.run_id,
            'experiment_id': run.info.experiment_id,
            'rmse': metrics['rmse'],
            'status': run.info.status,
            'start_time': run.info.start_time,
            'end_time': run.info.end_time,
            'ordered_by':["metrics.rmse ASC"],
        })
print(len(filtered_runs)) # 5 best options

# Convert the filtered runs to a DataFrame
filtered_runs_df = pd.DataFrame(filtered_runs)

5


In [40]:
filtered_runs_df.head()

,run_id,experiment_id,rmse,status,start_time,end_time,ordered_by
0,0442684afe664b3c9e2f25a43f74ebab,1,2.987523,FINISHED,1721085449236,1721085452838,[metrics.rmse ASC]
1,428efe5421a94c7d8dc1c8624af709cc,1,2.987182,FINISHED,1721085440426,1721085443651,[metrics.rmse ASC]
2,cce49f52241f464aa5f21691ecf7f2c7,1,2.984167,FINISHED,1721085380614,1721085383443,[metrics.rmse ASC]
3,ba7fc5810a2d4215a8556ff885afca27,1,2.986388,FINISHED,1721085370712,1721085374367,[metrics.rmse ASC]
4,d2c5d41c65524388bdd18e8ad385317d,1,2.975468,FINISHED,1721085349944,1721085353580,[metrics.rmse ASC]
